<h1>Proiect: Tabelare și data visualizatio

Student: Guliciuc George Gabriel

Grupa: mDM11

Context:

    Este deja cunoscut că mărimea și zona în care se află  influnteaza prețul unui apartament, prin acest proiect doresc să văd dacă există o relație între prețul apartamentul și alte specificații a acestuia (anul construcției, etajul la care se află apartamentul, înălțimea blocului, tipul de încălzirea, dacă are un loc de parcare/garaj) în zona metropolitană Iași.

In [53]:
import pandas as pd
import numpy as np
from Functionalitati import *

<h2>Prezentarea bazei de date inițiale

Pentru acest proiect am extras datele de la anuturile de vânzare din zona metropolitană Iași de pe situl https://www.storia.ro/ro/rezultate/vanzare/apartament/iasi. Folosind un proces de data scraping, algoritmul se afla in fisierul "DataScraping.py". Datele extrase au fost salvate in fisierul "apartamente_de_vanzare.xlsx".

Am ales să folosesc datele doar acest site deoarece majoritatea anunțurilor se repetau și pe celalalte situri.

In [54]:
vanzare_df = pd.read_excel("apartamente_de_vanzare.xlsx")
inchiriere_df = pd.read_excel("apartamente_de_inchiriat.xlsx")

Baza de date inițială conține mai multe coloane, majoritatea nu ne sunt utile pentru acest proiect și o să fie eliminate. De asemenea toate datele sunt de tipul "strîng/text"(Python o sa foloseaca tipul "object" pt string https://stackoverflow.com/questions/21018654/strings-in-a-dataframe-but-dtype-is-object ), mai putin coloana "Numărul de camere" care este "int/numar" 

In [55]:
vanzare_df.dtypes

Zona                       object
Pret                       object
Suprafață                  object
Forma de proprietate       object
Numărul de camere           int64
Stare                      object
Etaj                       object
Balcon/grădină/terasă      object
Chirie                     object
Garaj/loc de parcare       object
Vizionare la distanță      object
Încălzire                  object
Tip proprietate            object
Tip vânzător               object
Liber de la                object
Anul construcției          object
Tip clădire                object
Tip geamuri                object
Lift                       object
Media                      object
Siguranță                  object
Facilități                 object
Informații suplimentare    object
Material de construcție    object
Titlu                      object
Descriere                  object
dtype: object

<h3> Din baza inițială o să salvăm următoarele coloane:</h3>

- Zona: Specifică locul unde se afla blocul;

- Pret: Prețul apartamentului în euro (sunt câteva cazuri în care este în RON);

- Suprafață: Suprafață utilă în m²;

- Garaj/loc de parcare: Garaj/loc de parcare sau câmp gol

- Încălzire: Modul de încălzirea a apartamentului sau câmp gol

- Anul construcției: Anul în care lucrările au fost finalizate

<h3> Note: </h3> O să ne folosim și de coloana "Descriere" pentru a completă câmpurile goale. În multe anunțuri unde anumite câmpuri sunt goale( ex: "Etaj")informațiile care ar trebui să umple respectivele câmpuri se pot găsi în "Descriere".

<h2> Operațiuni preliminare </h2>
<h3> Note: </h3>
Pentru nu a încarcă acest fișier, funcțiile folosite au fost scrise în fișierul "Functionalitati.py".

<h3>Câmpurile goale:</h3>
În datele inițiale intrările goale sunt "umplute"  cu "Cere informații" și "fără informații", pentru a nu există 2 lucruri care de fapt reprezintă același lucru le-am înlocuit cu NaN

In [56]:
vanzare_df.replace("Cere informații", np.nan, inplace=True)
vanzare_df.replace("fără informații", np.nan, inplace=True)

In [57]:
inchiriere_df.replace("Cere informații", np.nan, inplace=True)
inchiriere_df.replace("fără informații", np.nan, inplace=True)

<h3> Coloana "Pret":</h3>

Prețul este un strîng, că să îl putem transformă în valoare numerică vom fi nevoiți să:

- să eliminăm caracterul "€"

- să eliminăm spațiile

În cazul prețurilor în "RON" ele o să fie convertite în euro, am folosit rată de schimb 5 RON = 1 Euro

In [58]:
vanzare_df['Pret'].head()

0    168 000 €
1    128 000 €
2    396 000 €
3    102 405 €
4    120 000 €
Name: Pret, dtype: object

In [59]:
vanzare_df['Pret'] = vanzare_df['Pret'].apply(convert_money)
inchiriere_df['Pret'] = inchiriere_df['Pret'].apply(convert_money)

In [60]:
vanzare_df['Pret'].head()

0    168000
1    128000
2    396000
3    102405
4    120000
Name: Pret, dtype: int64

<h3> Coloana "Suprafață":</h3>

Suprafață este un strîng, că să îl putem transformă în valoare numerică vom nevoiți să:

- să eliminăm caracterul "m²"

- să eliminăm spațiile
- să schimbăm caracterul "," cu "."

In [62]:
vanzare_df['Suprafață'].head()

0    66,18 m²
1       81 m²
2      230 m²
3    68,27 m²
4       78 m²
Name: Suprafață, dtype: object

In [63]:
vanzare_df['Suprafață'] = vanzare_df['Suprafață'].apply(convert_area)
inchiriere_df['Suprafață'] = inchiriere_df['Suprafață'].apply(convert_area)

In [65]:
vanzare_df['Suprafață'].head()

0     66.18
1     81.00
2    230.00
3     68.27
4     78.00
Name: Suprafață, dtype: float64

<h3> Coloana "Etaj":</h3>
Această coloana conține și spații goale, în primult completăm( unde se poate) cu informațiile din coloana "Descriere"( prin parsarea textului). De asemenea în această coloana trecute și etajul apartamentului cât și numărul de etaje ale blocului, de aceea o să creem două coloane noi:

- 'Etaj apartament': Care o să conțină etajul la care se află apartamenul;

-  'Număr Etaj Bloc': Care o să conțină numărul de etaje ale blocului


In [7]:
vanzare_df['Etaj'] = vanzare_df.apply(lambda df: building_floors(df["Etaj"], df["Descriere"]), axis=1)
inchiriere_df['Etaj'] = inchiriere_df.apply(lambda df: building_floors(df["Etaj"], df["Descriere"]), axis=1)

In [8]:
vanzare_df['Etaj apartament'] = vanzare_df['Etaj'].apply(apartment_floor)
inchiriere_df['Etaj apartament'] = inchiriere_df['Etaj'].apply(apartment_floor)

In [9]:
vanzare_df['Numar Etaj Bloc'] = vanzare_df['Etaj'].apply(number_of_floors)
inchiriere_df['Numar Etaj Bloc'] = inchiriere_df['Etaj'].apply(number_of_floors)

In [10]:
vanzare_df["Anul construcției"] = vanzare_df.apply(lambda df: get_year(df["Anul construcției"], df["Descriere"]), axis=1)
inchiriere_df["Anul construcției"] = inchiriere_df.apply(lambda df: get_year(df["Anul construcției"], df["Descriere"]), axis=1)

In [11]:
vanzare_df['Garaj/loc de parcare'] = vanzare_df.apply(lambda df: parcare_or_garaj(df['Garaj/loc de parcare'], df["Descriere"]), axis=1)
inchiriere_df['Garaj/loc de parcare'] = inchiriere_df.apply(lambda df: parcare_or_garaj(df['Garaj/loc de parcare'], df["Descriere"]), axis=1)

In [12]:
vanzare_df['Garaj/loc de parcare'].replace(np.nan, "Fara loc", inplace=True)
inchiriere_df['Garaj/loc de parcare'].replace(np.nan, "Fara loc", inplace=True)

In [13]:
new_vanzare_df = vanzare_df.loc[:,("Zona", "Pret", 'Suprafață', 'Numărul de camere', 'Anul construcției',
                                   'Etaj apartament', 'Numar Etaj Bloc', 'Garaj/loc de parcare')]
new_inchiriere_df = inchiriere_df.loc[:, ("Zona", "Pret", 'Suprafață', 'Numărul de camere', 'Anul construcției',
                                          'Etaj apartament', 'Numar Etaj Bloc', 'Garaj/loc de parcare')]

In [14]:
new_vanzare_df['Anul construcției'].replace(np.nan, 0, inplace=True)
new_vanzare_df['Anul construcției'] = new_vanzare_df['Anul construcției'].astype('int')
new_inchiriere_df["Anul construcției"].replace(np.nan, 0, inplace=True)
new_inchiriere_df["Anul construcției"] = new_inchiriere_df["Anul construcției"].astype("int")

In [15]:
new_vanzare_df['Numar Etaj Bloc'].replace(np.nan, 0, inplace=True)
index_to_drop = new_vanzare_df.index[new_vanzare_df['Numar Etaj Bloc'] == "24"].to_list()
new_vanzare_df.drop(index_to_drop, inplace=True)

In [16]:
new_inchiriere_df['Numar Etaj Bloc'].replace(np.nan, 0, inplace=True)
index_to_drop = new_inchiriere_df.index[new_inchiriere_df['Numar Etaj Bloc'] == "24"].to_list()
new_inchiriere_df.drop(index_to_drop, inplace=True)

In [17]:
new_vanzare_df['Etaj apartament'].replace(np.nan, 0, inplace=True)
index_to_drop = new_vanzare_df[new_vanzare_df['Etaj apartament'] == 0].index
new_vanzare_df.drop(index_to_drop, inplace=True)

In [18]:
new_inchiriere_df['Etaj apartament'].replace(np.nan, 0, inplace=True)
index_to_drop = new_inchiriere_df[new_inchiriere_df['Etaj apartament'] == 0].index
new_inchiriere_df.drop(index_to_drop, inplace=True)

In [19]:
mode_mansarda = int(new_vanzare_df[new_vanzare_df['Etaj apartament'] == 'mansardă']['Numar Etaj Bloc'].mode().to_list()[0])
new_vanzare_df['Numar Etaj Bloc'] = new_vanzare_df.apply(lambda df:
                                                         set_max_floor(df['Etaj apartament'], df['Numar Etaj Bloc'], mode_mansarda), axis=1)

In [20]:
mode_mansarda = int(new_inchiriere_df[new_inchiriere_df['Etaj apartament'] == 'mansardă']['Numar Etaj Bloc'].mode().to_list()[0])
new_inchiriere_df['Numar Etaj Bloc'] = new_inchiriere_df.apply(lambda df:
                                                         set_max_floor(df['Etaj apartament'], df['Numar Etaj Bloc'], mode_mansarda), axis=1)

In [21]:
new_vanzare_df['Zona Oras'] = new_vanzare_df['Zona'].apply(city_zone)
new_inchiriere_df['Zona Oras'] = new_inchiriere_df['Zona'].apply(city_zone)

In [22]:
new_vanzare_df["Anul construcției"] = new_vanzare_df.apply(lambda df:
                                                              year_mode(df['Zona Oras'], df["Anul construcției"], new_vanzare_df), axis=1)
new_inchiriere_df["Anul construcției"] = new_inchiriere_df.apply(lambda df:
                                                              year_mode(df['Zona Oras'], df["Anul construcției"], new_inchiriere_df), axis=1)

In [23]:
new_vanzare_df['Cutremur'] = new_vanzare_df.apply(lambda df:
                                                  dangerous_building(df['Anul construcției'], df['Numar Etaj Bloc']), axis=1)
new_inchiriere_df['Cutremur'] = new_inchiriere_df.apply(lambda df:
                                                  dangerous_building(df['Anul construcției'], df['Numar Etaj Bloc']), axis=1)


In [24]:
new_vanzare_df['Anul construcției'].unique()

array([2015, 1989, 2001, 2023, 2022, 1970, 2026, 2018, 1997,    0, 2008,
       1990, 2017, 2021, 2019, 1972, 1965, 2025, 2024, 1980, 1978, 2020,
       1979, 2016, 1986, 1968, 2010, 1977, 1985, 1992, 1976, 1973, 1974,
       1984, 1975, 1967, 1988, 1983, 1987, 1981, 1966, 2009, 2014, 1971,
       1995, 1969, 1962, 1982, 2005, 1955, 2011, 1960, 2013, 2012, 1964,
       1993, 1963, 1958, 1996, 1959, 2000, 1954, 1991, 1890, 1998, 1999,
       1994, 1952, 1950, 1961, 2006], dtype=int64)